In [1]:
import numpy as np
import pandas as pd
from google.colab import drive

In [2]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


In [3]:
zipcode_path = '/content/drive/My Drive/CIS550/zip_code_database.xlsx' # path to demographics dataset
demographics_path = '/content/drive/My Drive/CIS550/cc-est2019-alldata.csv' # path to zip code dataset

In [4]:
zip_data = pd.read_excel(zipcode_path) # import zip code data
dem_data = pd.read_csv(demographics_path, encoding = "ISO-8859-1") # import demographic data

In [5]:
zip_data.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631.0,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631.0,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,787939.0,NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,787939.0,NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787.0,NaN,US,18.43,-67.15,0


In [6]:
print(zip_data.dtypes)
print(zip_data.shape)

zip                                int64
type                              object
decommissioned                     int64
primary_city                      object
acceptable_cities                 object
unacceptable_cities               object
state                             object
county                            object
timezone                          object
area_codes                       float64
world_region                      object
country                           object
latitude                         float64
longitude                        float64
irs_estimated_population_2015      int64
dtype: object
(42632, 15)


In [7]:
dem_data.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,IA_MALE,IA_FEMALE,AA_MALE,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,WAC_MALE,WAC_FEMALE,BAC_MALE,BAC_FEMALE,IAC_MALE,IAC_FEMALE,AAC_MALE,AAC_FEMALE,NAC_MALE,NAC_FEMALE,NH_MALE,NH_FEMALE,NHWA_MALE,NHWA_FEMALE,NHBA_MALE,NHBA_FEMALE,NHIA_MALE,NHIA_FEMALE,NHAA_MALE,NHAA_FEMALE,NHNA_MALE,NHNA_FEMALE,NHTOM_MALE,NHTOM_FEMALE,NHWAC_MALE,NHWAC_FEMALE,NHBAC_MALE,NHBAC_FEMALE,NHIAC_MALE,NHIAC_FEMALE,NHAAC_MALE,NHAAC_FEMALE,NHNAC_MALE,NHNAC_FEMALE,H_MALE,H_FEMALE,HWA_MALE,HWA_FEMALE,HBA_MALE,HBA_FEMALE,HIA_MALE,HIA_FEMALE,HAA_MALE,HAA_FEMALE,HNA_MALE,HNA_FEMALE,HTOM_MALE,HTOM_FEMALE,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,21295,22002,4559,5130,119,139,200,284,29,18,367,429,21633,22391,4704,5306,277,314,300,409,42,37,25875,27386,20709,21485,4512,5091,103,115,194,280,13,9,344,406,21026,21853,4647,5258,251,282,291,398,23,27,694,616,586,517,47,39,16,24,6,4,16,9,23,23,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,1411,1316,362,317,5,3,13,15,1,0,74,62,1479,1368,405,362,23,18,34,28,3,1,1778,1651,1337,1260,356,313,2,2,13,15,0,0,70,61,1402,1312,396,357,19,17,34,28,1,0,88,62,74,56,6,4,3,1,0,0,1,0,4,1,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,1521,1526,399,374,14,8,17,21,1,3,49,58,1570,1583,425,403,27,19,32,42,3,4,1933,1916,1460,1465,398,372,12,2,17,21,0,3,46,53,1506,1517,423,400,25,12,30,39,1,4,68,74,61,61,1,2,2,6,0,0,1,0,3,5,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,1658,1620,431,406,15,12,23,18,4,1,40,62,1694,1681,453,436,29,27,32,37,4,5,2105,2055,1613,1570,421,403,12,9,22,18,3,0,34,55,1643,1624,440,429,24,22,30,36,3,4,66,64,45,50,10,3,3,3,1,0,1,1,6,7,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,1628,1585,502,424,12,7,25,14,4,2,42,45,1664,1624,525,444,23,20,39,31,6,5,2153,2026,1580,1543,495,420,12,5,23,14,1,1,42,43,1616,1580,518,439,23,18,37,30,3,4,60,51,48,42,7,4,0,2,2,0,3,1,0,2,48,44,7,5,0,2,2,1,3,1


In [8]:
print(dem_data.dtypes)
print(dem_data.shape)

SUMLEV          int64
STATE           int64
COUNTY          int64
STNAME         object
CTYNAME        object
                ...  
HIAC_FEMALE     int64
HAAC_MALE       int64
HAAC_FEMALE     int64
HNAC_MALE       int64
HNAC_FEMALE     int64
Length: 80, dtype: object
(716376, 80)


In [9]:
# Check which columns in the zip code data contain null values. 
zip_data.isnull().sum().sort_values(ascending = False)

world_region                     42299
acceptable_cities                33609
unacceptable_cities              30314
area_codes                        2934
county                             842
timezone                           714
irs_estimated_population_2015        0
longitude                            0
latitude                             0
country                              0
state                                0
primary_city                         0
decommissioned                       0
type                                 0
zip                                  0
dtype: int64

In [10]:
# Based on the result above, can drop world_region, acceptable_cities, unacceptable_cities, area_codes, and timezone columns since they are not used
zip_data = zip_data.drop(columns=['world_region', 'acceptable_cities', 'unacceptable_cities', 'area_codes', 'timezone'])
zip_data.head()

,zip,type,decommissioned,primary_city,state,county,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,PR,Adjuntas Municipio,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,PR,Aguada Municipio,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,PR,Aguadilla Municipio,US,18.43,-67.15,0


In [11]:
# Check how many counties in the demographic data don't exist in the zip code data
dem_data = dem_data.assign(CountyExistsInZipTable=dem_data.CTYNAME.isin(zip_data.county))
dem_data_unmatched_zips = dem_data.loc[dem_data['CountyExistsInZipTable'] == False]
dem_data_unmatched_zips = dem_data_unmatched_zips[['STNAME', 'CTYNAME']]
dem_data_unmatched_zips.value_counts(['STNAME', 'CTYNAME']) #counties in the demographic data don't exist in the zip code data

STNAME        CTYNAME             
Virginia      Martinsville city       228
              Manassas Park city      228
              Lexington city          228
              Fairfax city            228
              Emporia city            228
              Covington city          228
South Dakota  Oglala Lakota County    228
Louisiana     LaSalle Parish          228
Alaska        Petersburg Borough      228
              Kusilvak Census Area    228
dtype: int64

In [12]:
# Capitalize the first letter of each word in the county names in both the zip code and demographic tables
zip_data['county'] = zip_data['county'].str.title()
dem_data['CTYNAME'] = dem_data['CTYNAME'].str.title()

In [13]:
# Recheck how many counties in the demographic data don't exist in the zip code data
dem_data = dem_data.assign(CountyExistsInZipTable=dem_data.CTYNAME.isin(zip_data.county))
dem_data_unmatched_zips = dem_data.loc[dem_data['CountyExistsInZipTable'] == False]
dem_data_unmatched_zips = dem_data_unmatched_zips[['STNAME', 'CTYNAME']]
dem_data_unmatched_zips.value_counts(['STNAME', 'CTYNAME']) #counties in the demographic data don't exist in the zip code data

STNAME        CTYNAME             
Virginia      Manassas Park City      228
              Lexington City          228
              Emporia City            228
              Covington City          228
South Dakota  Oglala Lakota County    228
Louisiana     Lasalle Parish          228
Alaska        Petersburg Borough      228
              Kusilvak Census Area    228
dtype: int64

In [14]:
# Check how these questionable counties appear in the zip code data
contains_Manassas = zip_data['county'].str.contains('Manassas')
contains_Lexington = zip_data['county'].str.contains('Lexington')
contains_Emporia = zip_data['county'].str.contains('Emporia')
contains_Covington = zip_data['county'].str.contains('Covington')
contains_Oglala = zip_data['county'].str.contains('Oglala')
contains_Salle = zip_data['county'].str.contains('Salle')
contains_Petersburg = zip_data['county'].str.contains('Petersburg')
contains_Kusilvak = zip_data['county'].str.contains('Kusilvak')

search_filter = contains_Manassas | contains_Lexington | contains_Emporia | contains_Covington | contains_Oglala | contains_Salle | contains_Petersburg | contains_Kusilvak
zip_data_filtered = zip_data.loc[search_filter, ['state', 'county']]
zip_data_filtered.value_counts()

state  county                
SC     Lexington County          16
AL     Covington County          11
LA     La Salle Parish            5
TX     La Salle County            4
VA     Petersburg City            3
       Manassas City              3
MS     Covington County           3
AK     Petersburg Census Area     3
IL     La Salle County            1
dtype: int64

In [15]:
# Check how these questionable counties appear in the demographic code data
contains_Manassas_dem = dem_data['CTYNAME'].str.contains('Manassas')
contains_Lasalle_dem = dem_data['CTYNAME'].str.contains('Lasalle')
contains_Petersburg_dem = dem_data['CTYNAME'].str.contains('Petersburg')

search_filter_dem = contains_Manassas_dem | contains_Lasalle_dem | contains_Petersburg_dem
dem_data_filtered = dem_data.loc[search_filter_dem, ['STNAME', 'CTYNAME']]
dem_data_filtered.value_counts()

STNAME     CTYNAME           
Virginia   Petersburg City       228
           Manassas Park City    228
           Manassas City         228
Louisiana  Lasalle Parish        228
Illinois   Lasalle County        228
Alaska     Petersburg Borough    228
dtype: int64

In [16]:
# Replace certain counties in demographic data to align with formatting of zip code data
dem_data['CTYNAME'] = dem_data['CTYNAME'].str.replace('Lasalle Parish','La Salle Parish')
dem_data['CTYNAME'] = dem_data['CTYNAME'].str.replace('Petersburg Borough','Petersburg Census Area')

# Recheck how many counties in the demographic data don't exist in the zip code data
dem_data = dem_data.assign(CountyExistsInZipTable=dem_data.CTYNAME.isin(zip_data.county))
dem_data_unmatched_zips = dem_data.loc[dem_data['CountyExistsInZipTable'] == False]
dem_data_unmatched_zips = dem_data_unmatched_zips[['STNAME', 'CTYNAME']]
dem_data_unmatched_zips.value_counts(['STNAME', 'CTYNAME']) #counties in the demographic data don't exist in the zip code data

STNAME        CTYNAME             
Virginia      Manassas Park City      228
              Lexington City          228
              Emporia City            228
              Covington City          228
South Dakota  Oglala Lakota County    228
Alaska        Kusilvak Census Area    228
dtype: int64

In [17]:
# Check how many counties in the zip code data don't exist in the demographic data
zip_data = zip_data.assign(CountyExistsInDemTable=zip_data.county.isin(dem_data.CTYNAME))
zip_data_unmatched_zips = zip_data.loc[zip_data['CountyExistsInDemTable'] == False]
zip_data_unmatched_zips = zip_data_unmatched_zips[['state', 'county']]
zip_data_unmatched_zips.value_counts(['state', 'county']) #counties in the zip code data don't exist in the demographic data

state  county                   
PR     San Juan Municipio           18
AK     Municipality Of Anchorage    12
       Wade Hampton Census Area     12
PR     San Juan                      9
       Ponce Municipio               7
                                    ..
       Manatí Municipio              1
       Maricao Municipio             1
       Maunabo Municipio             1
       Moca Municipio                1
       Guayanilla Municipio          1
Length: 105, dtype: int64

In [18]:
# Delete rows with Federal districts (aside from Washington DC)
federal_indexes = zip_data[(zip_data['state'] == 'AS') | (zip_data['state'] == 'GU') | (zip_data['state'] == 'MP') |
                           (zip_data['state'] == 'PR') | (zip_data['state'] == 'VI') | (zip_data['state'] == 'FM') |
                           (zip_data['state'] == 'MH') | (zip_data['state'] == 'PW')].index  

# Delete rows with 0 latitude and 0 longitude values
lat0_lon0_indexes = zip_data[(zip_data['latitude'] == 0) & (zip_data['longitude'] == 0)].index 

zip_data.drop(federal_indexes, inplace=True)
zip_data.drop(lat0_lon0_indexes, inplace=True)

print(zip_data.shape)

(41702, 11)


In [19]:
# Recheck how many counties in the zip code data don't exist in the demographic data
zip_data = zip_data.assign(CountyExistsInDemTable=zip_data.county.isin(dem_data.CTYNAME))
zip_data_unmatched_zips = zip_data.loc[zip_data['CountyExistsInDemTable'] == False]
zip_data_unmatched_zips = zip_data_unmatched_zips[['state', 'county']]
zip_data_unmatched_zips.value_counts(['state', 'county']) #counties in the zip code data don't exist in the demographic data

state  county                                 
AK     Municipality Of Anchorage                  12
       Wade Hampton Census Area                   12
       City And Borough Of Juneau                  6
MN     St Louis County                             4
NY     St Lawrence County                          4
LA     St Mary Parish                              3
IN     La Porte County                             2
NM     Dona Ana County                             2
AK     Prince Of Wales-Outer Ketchikan Borough     2
NM     Doã±A Ana County                            2
MO     St Louis County                             1
       St Francois County                          1
NH     Coã¶S County                                1
MD     St Mary'S County                            1
LA     St Tammany Parish                           1
VA     Charles City                                1
LA     St Martin Parish                            1
       St Landry Parish                            1

In [20]:
# Replace "St" with "St." in the zip code county data
zip_data['county'] = zip_data['county'].str.replace('St ','St. ')

In [21]:
# Recheck how many counties in the zip code data don't exist in the demographic data
zip_data = zip_data.assign(CountyExistsInDemTable=zip_data.county.isin(dem_data.CTYNAME))
zip_data_unmatched_zips = zip_data.loc[zip_data['CountyExistsInDemTable'] == False]
zip_data_unmatched_zips = zip_data_unmatched_zips[['state', 'county']]
zip_data_unmatched_zips.value_counts(['state', 'county']) #counties in the zip code data don't exist in the demographic data

state  county                                 
AK     Wade Hampton Census Area                   12
       Municipality Of Anchorage                  12
       City And Borough Of Juneau                  6
NM     Doã±A Ana County                            2
       Dona Ana County                             2
IN     La Porte County                             2
AK     Prince Of Wales-Outer Ketchikan Borough     2
VA     James City                                  1
       Charles City                                1
NH     Coã¶S County                                1
dtype: int64

In [22]:
# Check how these questionable counties appear in the demographic data
contains_Wade = dem_data['CTYNAME'].str.contains('Wade')
contains_Anchorage = dem_data['CTYNAME'].str.contains('Anchorage')
contains_Juneau = dem_data['CTYNAME'].str.contains('Juneau')
contains_Ana = dem_data['CTYNAME'].str.contains('Ana')
contains_Porte = dem_data['CTYNAME'].str.contains('Porte')
contains_Wales = dem_data['CTYNAME'].str.contains('Wales')
contains_James = dem_data['CTYNAME'].str.contains('James')
contains_Charles = dem_data['CTYNAME'].str.contains('Charles')
contains_Coa = dem_data['CTYNAME'].str.contains('Coa')

search_filter = contains_Wade | contains_Anchorage | contains_Juneau | contains_Ana | contains_Porte | contains_Wales | contains_James | contains_Charles | contains_Coa
dem_data_filtered = dem_data.loc[search_filter, ['STNAME', 'CTYNAME']]
dem_data_filtered.value_counts()

STNAME          CTYNAME                          
Wisconsin       Juneau County                        228
Minnesota       Wadena County                        228
Alaska          Juneau City And Borough              228
                Prince Of Wales-Hyder Census Area    228
Indiana         Porter County                        228
Louisiana       St. Charles Parish                   228
                St. James Parish                     228
Maryland        Charles County                       228
Mississippi     Coahoma County                       228
Virginia        James City County                    228
Missouri        St. Charles County                   228
New Mexico      Doña Ana County                      228
Oklahoma        Coal County                          228
South Carolina  Charleston County                    228
South Dakota    Charles Mix County                   228
Virginia        Charles City County                  228
Alaska          Anchorage Municipality

In [23]:
# Check how these questionable counties appear in the zip code data
contains_Wade = zip_data['county'].str.contains('Wade')
contains_Anchorage = zip_data['county'].str.contains('Anchorage')
contains_Juneau = zip_data['county'].str.contains('Juneau')
contains_Ana = zip_data['county'].str.contains('Ana')
contains_Porte = zip_data['county'].str.contains('Porte')
contains_Wales = zip_data['county'].str.contains('Wales')
contains_James = zip_data['county'].str.contains('James City')
contains_Charles = zip_data['county'].str.contains('Charles City')
contains_Coajs = zip_data['county'].str.contains('Coã¶S')

zip_data = zip_data.assign(latitude_rounded=zip_data.latitude.round(decimals=0)) #Add a field with rounded latitude values
zip_data = zip_data.assign(longitude_rounded=zip_data.longitude.round(decimals=0)) #Add a field with rounded longitude values

search_filter = contains_Wade | contains_Anchorage | contains_Juneau | contains_Ana | contains_Porte | contains_Wales | contains_James | contains_Charles | contains_Coajs
zip_data_filtered = zip_data.loc[search_filter, ['state', 'county', 'latitude_rounded', 'longitude_rounded', 'irs_estimated_population_2015']]
zip_data_filtered = zip_data_filtered.groupby(['state','county', 'latitude_rounded', 'longitude_rounded']).sum()
zip_data_filtered.head(50)

irs_estimated_population_2015
state county                                  latitude_rounded longitude_rounded                               
AK    Anchorage Municipality                  61.0             -150.0                                    228760
                                                               -149.0                                     37187
      City And Borough Of Juneau              58.0             -135.0                                      4332
                                                               -134.0                                      3925
      Juneau City And Borough                 58.0             -134.0                                      1930
                                              59.0             -135.0                                     21730
      Municipality Of Anchorage               56.0             -159.0                                        23
                                              61.0             -150.0                                     18436
      Prince Of Wales-Hyder Census Area       55.0             -133.0                                      1759
                                              56.0             -134.0                                        48
                                                               -133.0                                       882
                                                               -132.0                                       517
                                                               -130.0                                        85
      Prince Of Wales-Outer Ketchikan Borough 55.0             -132.0                                       262
                                                               -131.0                                      1140
      Wade Hampton Census Area                62.0             -166.0                                      2268
                                                               -165.0                                       140
                                                               -164.0                                       687
                                                               -163.0                                      1130
                                                               -162.0                                       350
                                                               -161.0                                       274
                                              63.0             -165.0                                      1272
                                                               -164.0                                       530
IN    La Porte County                         42.0             -87.0                                       1653
      Porter County                           41.0             -87.0                                      88857
                                              42.0             -87.0                                      61095
MN    Wadena County                           46.0             -95.0                                       7600
                                              47.0             -95.0                                       6330
NH    Coã¶S County                            44.0             -71.0                                          0
NM    Dona Ana County                         32.0             -107.0                                      1368
      Doã±A Ana County                        32.0             -107.0                                      1348
      Doña Ana County                         32.0             -107.0                                    147388
                                                               -106.0                                       660
                                              33.0             -107.0                                     29110
VA    Charles City                            37.0

In [24]:
# Replace certain counties in zip code data to align with formatting of demographic data, since these counties have similar
# longitude and latitude coordinates as countise that already exist in the demographic data
zip_data['county'] = zip_data['county'].str.replace('Municipality Of Anchorage','Anchorage Municipality')
zip_data['county'] = zip_data['county'].str.replace('City And Borough Of Juneau','Juneau City And Borough')
zip_data['county'] = zip_data['county'].str.replace('Doã±A Ana County','Doña Ana County')
zip_data['county'] = zip_data['county'].str.replace('Dona Ana County','Doña Ana County')
zip_data['county'] = zip_data['county'].str.replace('La Porte County','Porter County')
zip_data['county'] = zip_data['county'].str.replace('Prince Of Wales-Outer Ketchikan Borough','Prince Of Wales-Hyder Census Area')
zip_data['county'] = zip_data['county'].str.replace('La Porte County','Porter County')
zip_data['county'] = zip_data['county'].str.replace(r'^James City$', 'James City County')
zip_data['county'] = zip_data['county'].str.replace(r'^Charles City$','Charles City County')

# Delete "Coã¶S County" rows since it has no population
coajs_indexes = zip_data[zip_data['county'] == 'Coã¶S County'].index
zip_data.drop(coajs_indexes, inplace=True)
print(zip_data.shape)

(41701, 13)


In [25]:
# Recheck how many counties in the zip code data don't exist in the demographic data
zip_data = zip_data.assign(CountyExistsInDemTable=zip_data.county.isin(dem_data.CTYNAME))
zip_data_unmatched_zips = zip_data.loc[zip_data['CountyExistsInDemTable'] == False]
zip_data_unmatched_zips = zip_data_unmatched_zips[['state', 'county']]
zip_data_unmatched_zips.value_counts(['state', 'county']) #counties in the zip code data don't exist in the demographic data

state  county                  
AK     Wade Hampton Census Area    12
dtype: int64

In [26]:
# Drop the temporary processing fields from the zip code table
zip_data = zip_data.drop(columns=['latitude_rounded', 'longitude_rounded', 'CountyExistsInDemTable'])
zip_data.head(50)

,zip,type,decommissioned,primary_city,state,county,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NY,Suffolk County,US,40.81,-73.04,0
194,1001,STANDARD,0,Agawam,MA,Hampden County,US,42.06,-72.61,15220
195,1002,STANDARD,0,Amherst,MA,Hampshire County,US,42.37,-72.52,16570
196,1003,PO BOX,0,Amherst,MA,Hampshire County,US,42.39,-72.52,184
197,1004,PO BOX,0,Amherst,MA,Hampshire County,US,42.37,-72.52,794
198,1005,STANDARD,0,Barre,MA,Worcester County,US,42.42,-72.10,4270
199,1007,STANDARD,0,Belchertown,MA,Hampshire County,US,42.27,-72.40,14180
200,1008,STANDARD,0,Blandford,MA,Hampden County,US,42.18,-72.93,1140
201,1009,PO BOX,0,Bondsville,MA,Hampden County,US,42.20,-72.34,1238


In [27]:
# Shorten the demographics table to only the fields needed

# Delete the rows with an age group of 0
age_group_0_indexes = dem_data[dem_data['AGEGRP'] == 0].index
dem_data.drop(age_group_0_indexes, inplace=True)

# Bucket the age groups - 0-19, 20-39, 40-59, 60+
age_bins = [0, 4, 8, 12, 18]
age_labels = ['0-19', '20-39', '40-59', '60+']
dem_data['AGE_BIN'] = pd.cut(x=dem_data['AGEGRP'], bins=age_bins, labels=age_labels)

# Drop the excess fields and temporary processing field
dem_data = dem_data[['STNAME', 'CTYNAME', 'YEAR', 'AGE_BIN', 'TOT_POP', 'TOT_MALE', 'TOT_FEMALE',
                     'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 
                     'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE',
                     'NH_MALE', 'NH_FEMALE', 'H_MALE', 'H_FEMALE']].copy()

# Delete the past year rows (only keep the current year) and then drop the year column (since they're all the same year)
past_year_indexes = dem_data[dem_data['YEAR'] < 12].index
dem_data.drop(past_year_indexes, inplace=True)
dem_data.drop(columns=['YEAR'], inplace=True)

# Group by the age buckets
dem_data = pd.pivot_table(dem_data, index=['STNAME', 'CTYNAME', 'AGE_BIN'], values=['TOT_POP', 'TOT_MALE', 'TOT_FEMALE',
                     'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 
                     'AA_MALE', 'AA_FEMALE', 'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE',
                     'NH_MALE', 'NH_FEMALE', 'H_MALE', 'H_FEMALE'], aggfunc='sum')

dem_data.reset_index(level=[0, 1, 2], inplace=True)

In [28]:
# Replace state names with abbreviations in the demographics table
state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

dem_data = dem_data.replace({'STNAME': state_abbrev})

In [29]:
# Export the final zip code and demographic tables 
zip_data.to_csv("zip_data.csv", index=False)
dem_data.to_csv("demographic_data.csv", index=False)

In [30]:
from google.colab import files
files.download('zip_data.csv') 
files.download("demographic_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>